In [ ]:
# cspell:disable
from __future__ import annotations

from os.path import dirname

import IPython.display import display
import polarization.lhcb
import sympy as sp
from polarization.amplitude import AmplitudeModel
from polarization.io import perform_cached_doit
from polarization.lhcb import load_model_builder, load_model_parameters
from polarization.lhcb.particle import load_particles

THIS_DIRECTORY = dirname("/mnt/c/dev/redeboer/benchmarks")
DATA_DIRECTORY = dirname(polarization.lhcb.__file__)

In [ ]:
def create_amplitude_model() -> AmplitudeModel:
    model_choice = 0
    model_file = f"{DATA_DIRECTORY}/model-definitions.yaml"
    particles = load_particles(f"{DATA_DIRECTORY}/particle-definitions.yaml")
    amplitude_builder = load_model_builder(model_file, particles, model_choice)
    imported_parameter_values = load_model_parameters(
        model_file, amplitude_builder.decay, model_choice
    )
    reference_subsystem = 1
    model = amplitude_builder.formulate(reference_subsystem)
    model.parameter_defaults.update(imported_parameter_values)
    return model


model = create_amplitude_model()

In [ ]:
print("Unfolding intensity expression")
unfolded_intensity_expr = perform_cached_doit(model.full_expression)
print(f"  → {sp.count_ops(unfolded_intensity_expr):,} operations")
print("Substituting kinematic variables")
angle_definitions = {k: v.doit() for k, v in model.variables.items()}
full_intensity_expr = unfolded_intensity_expr.xreplace(angle_definitions)
print(f"  → {sp.count_ops(full_intensity_expr):,} operations")

In [ ]:
remaining_symbols = full_intensity_expr.xreplace(
    model.parameter_defaults
).free_symbols
assert len(remaining_symbols) == 3

In [ ]:
x = sp.Symbol("x", positive=True)
sp.sin(2 * sp.acos(x))

In [ ]:
from ampform.sympy import PoolSum
from symplot import partial_doit

unfolded_wigner_d = partial_doit(model.intensity, PoolSum)
full_intensity_expr = partial_doit(unfolded_wigner_d, PoolSum)
# full_intensity_expr = unfolded_wigner_d.xreplace(model.amplitudes)

In [ ]:
from sympy.physics.quantum.spin import WignerD

simplified_expr = full_intensity_expr
i = 0
for node in sp.postorder_traversal(full_intensity_expr):
    if isinstance(node, WignerD):
        substituted_node = sp.expand_trig(node.doit().xreplace(model.variables))
        display(substituted_node)
        substitution = {node: substituted_node}
        simplified_expr = simplified_expr.xreplace(substitution)